In [1]:
#Imports
import requests

import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
#db = client.#

In [2]:
#key = 'F5FC7EFDA058ACFD73B657E00676FEC2'
#payload = {'key': key}
#api_url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/'
def get_match_history():
        """Fetch data from the API."""
        response = requests.get('https://api.opendota.com/api/proMatches')
        print(response)
        data = response.json()
        print(data)
        #self.next_sequence_number = data['_next_sequence_number']
        return data

In [3]:
x = get_match_history()

<Response [200]>
[{'match_id': 5450581495, 'duration': 3137, 'start_time': 1591123331, 'radiant_team_id': 7422789, 'radiant_name': 'HellRaisers', 'dire_team_id': 2163, 'dire_name': 'Team Liquid', 'leagueid': 12027, 'league_name': 'ESL One Birmingham 2020 Online powered by Intel', 'series_id': 451259, 'series_type': 1, 'radiant_score': 21, 'dire_score': 49, 'radiant_win': False}, {'match_id': 5450485396, 'duration': 2073, 'start_time': 1591119168, 'radiant_team_id': 7422789, 'radiant_name': 'HellRaisers', 'dire_team_id': 2163, 'dire_name': 'Team Liquid', 'leagueid': 12027, 'league_name': 'ESL One Birmingham 2020 Online powered by Intel', 'series_id': 451259, 'series_type': 1, 'radiant_score': 37, 'dire_score': 12, 'radiant_win': True}, {'match_id': 5450449906, 'duration': 2322, 'start_time': 1591117768, 'radiant_team_id': 7118032, 'radiant_name': 'Winstrike Team', 'dire_team_id': 7820540, 'dire_name': 'Team Unique', 'leagueid': 11796, 'league_name': 'Parimatch League Season 3', 'series_

In [4]:
y = [i['match_id'] for i in x]

In [16]:
y

[5450581495,
 5450485396,
 5450449906,
 5450379602,
 5450332626,
 5450308703,
 5450245044,
 5450184004,
 5450113195,
 5450061065,
 5450020255,
 5450015852,
 5449956482,
 5449922096,
 5449919699,
 5449900116,
 5449892977,
 5449869799,
 5449830847,
 5449829040,
 5449813596,
 5449806735,
 5449756388,
 5449744829,
 5449734818,
 5449709866,
 5449686587,
 5449677465,
 5449635341,
 5449625010,
 5449620942,
 5449583286,
 5449569784,
 5449540107,
 5449532848,
 5449531556,
 5449496600,
 5449472858,
 5449446679,
 5449446588,
 5449409755,
 5449387622,
 5449373749,
 5449363291,
 5449323875,
 5449313854,
 5449312328,
 5449294568,
 5448545961,
 5448423128,
 5448372917,
 5448306852,
 5448286326,
 5448237111,
 5448207969,
 5448177445,
 5448062880,
 5447992646,
 5447964890,
 5447925886,
 5447867826,
 5447864354,
 5447811671,
 5447715153,
 5447667077,
 5447620916,
 5447586578,
 5447564856,
 5447538078,
 5447529628,
 5447519774,
 5447476888,
 5447466274,
 5447432826,
 5447287781,
 5447251181,
 5447229811,

In [13]:
from datetime import datetime
ts = 1591119168
z = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

In [15]:
z.split()[0]

'2020-06-02'

In [ ]:
#start
#for runs
    #call pro matches api - if list of match id's use lowest match id as last id
    #get list of match id's
    #for each id
        #if id not in db
            #call match detail api
            #clean and add data to db
    
